### Carrega Lista de Processos, e baixa as decisões


In [1]:
# Importando as Bibliotecas
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import json
import pymongo
import yaml
from datetime import datetime

In [2]:
# Carrega Configurações do arquivo config
with open("../config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]


In [3]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]

In [4]:
# Coleções Mongo
col_processos = db['processo']
col_decisoes = db['decisoes']

In [5]:
#dados = pd.DataFrame(list(col_processos.find({'processo.cod_empresa':{'$in':['AZUL','GOL','TAM']}})))
dados = pd.DataFrame(list(col_processos.find({})))
df_processos = pd.json_normalize(dados['processo'])
df_processos.shape

(55994, 8)

In [8]:
dados_decisoes = pd.DataFrame(list(col_decisoes.find({})))
df_decisoes = pd.json_normalize(dados_decisoes['decisao'])
df_decisoes.shape

(22616, 11)

In [9]:
df_coleta = pd.merge(df_processos,df_decisoes,how="left",on="nro_processo")

In [10]:
df_coleta.shape

(55994, 18)

In [11]:
def pega_valores(soup,tag):
    valor = soup.select_one('strong:-soup-contains("'+ tag +'")')
    if valor: 
        valor = valor.parent.text.strip()
        valor = valor.replace(tag +':','').replace('\t','').replace('\n','').strip()
        return valor

0        0013181-02.2022.8.26.0001
1        1027826-15.2022.8.26.0001
2        1027076-13.2022.8.26.0001
3        0012409-39.2022.8.26.0001
4        1026245-62.2022.8.26.0001
                   ...            
55989    1009260-37.2022.8.26.0609
55990    1009082-88.2022.8.26.0609
55991    1001680-75.2022.8.26.0634
55992    1015433-29.2022.8.26.0625
55993    1007279-76.2022.8.26.0704
Name: nro_processo, Length: 32834, dtype: object

In [16]:
procs = df_coleta[df_coleta['texto_decisao'].isna() == True]['nro_processo']
len(procs)


32834

In [17]:
dados = []

for proc in procs:
    url = "https://esaj.tjsp.jus.br/cjpg/pesquisar.do?conversationId=&dadosConsulta.pesquisaLivre=&tipoNumero=UNIFICADO&numeroDigitoAnoUnificado=1003031-42.2022&foroNumeroUnificado=0001&dadosConsulta.nuProcesso=" + proc
    
    try: 
        html = requests.get(url).content
    except: 
        pass
    soup = BeautifulSoup(html, 'html.parser')

    aviso = soup.select_one('div:-soup-contains("Não foi encontrado nenhum")')

    if aviso: 
        print("Processo: " + proc + " sem resultados")
        
    else:
        print("Processo: " + proc + " com resultados")
        classe = pega_valores(soup,'Classe')
        assunto = pega_valores(soup,'Assunto')
        magistrado = pega_valores(soup,'Magistrado')
        comarca = pega_valores(soup,'Comarca')
        foro = pega_valores(soup,'Foro')
        vara = pega_valores(soup,'Vara')
        data_disp = pega_valores(soup,'Data de Disponibilização')
        texto_decisao = soup.find('div',attrs={'align':'justify','style':'display: none;'}).text.strip()

        dados.append([proc,classe,assunto,magistrado,comarca,foro,vara,data_disp,texto_decisao])


Processo: 0013181-02.2022.8.26.0001 sem resultados
Processo: 1027826-15.2022.8.26.0001 sem resultados
Processo: 1027076-13.2022.8.26.0001 com resultados
Processo: 0012409-39.2022.8.26.0001 sem resultados
Processo: 1026245-62.2022.8.26.0001 sem resultados
Processo: 1025380-39.2022.8.26.0001 sem resultados
Processo: 0011670-66.2022.8.26.0001 com resultados
Processo: 1024031-98.2022.8.26.0001 sem resultados
Processo: 0011194-28.2022.8.26.0001 com resultados
Processo: 1023709-78.2022.8.26.0001 sem resultados
Processo: 0010978-67.2022.8.26.0001 com resultados
Processo: 0010211-29.2022.8.26.0001 sem resultados
Processo: 1021510-83.2022.8.26.0001 sem resultados
Processo: 0009996-53.2022.8.26.0001 sem resultados
Processo: 1020375-36.2022.8.26.0001 com resultados
Processo: 0009489-92.2022.8.26.0001 com resultados
Processo: 1018538-43.2022.8.26.0001 sem resultados
Processo: 1017424-69.2022.8.26.0001 com resultados
Processo: 1016377-60.2022.8.26.0001 sem resultados
Processo: 0007676-30.2022.8.26.

In [18]:
df_decisoes = pd.DataFrame(dados,columns=["nro_processo","classe","assunto","magistrado","comarca","foro","vara","data_disp","texto_decisao"])
df_decisoes['instancia'] = "1ª Instância"


In [19]:
def carrega_decisao(linha):
    json_str = str(linha.to_json())
    json_str = '{"decisao":' + json_str + '}'
    col_decisoes.find_one_and_update({'decisao.nro_processo':linha['nro_processo']},{'$set':json.loads(json_str)},upsert=True)

In [20]:
for index,decisao in df_decisoes.iterrows(): 
    carrega_decisao(df_decisoes.loc[index])


In [21]:
termino = datetime.now().strftime("%d/%m/%y %H:%M:%S")
print("Acabou em: " + termino)

Acabou em: 02/10/22 23:00:06


In [15]:
print("----")

----
